## Get extrinsic params using robust calibration paper
Output: R, T between camera pairs

In [2]:
import sys

if '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/' not in sys.path:
  sys.path.append('/Users/damianstone/Documents/Code/tennis-project/post-triangulation/')

In [3]:
import cv2 as cv
import numpy as np
import glob
import json

BASE_PATH = '/Users/damianstone/Documents/Code/tennis-project/post-triangulation/'
IMAGE_BASE_PATH = f'{BASE_PATH}/images/'
INTRINSIC_PARAMS_FILE_NAME = 'intrinsic_params.json'

In [7]:
# load intrinsic parameters
with open(f'{BASE_PATH}/data/{INTRINSIC_PARAMS_FILE_NAME}', "r") as file:
    camera_data = json.load(file)

camera_data

{'CAM_1_LEFT': {'fx': 1800.0,
  'fy': 1805.0,
  'cx': 1920.0,
  'cy': 1080.0,
  'D': [-0.25, 0.05, 0.0, 0.0, -0.02],
  'K': [[1800.0, 0, 1920.0], [0, 1805.0, 1080.0], [0, 0, 1]]},
 'CAM_2_RIGHT': {'fx': 1795.0,
  'fy': 1800.0,
  'cx': 1918.0,
  'cy': 1078.0,
  'D': [-0.23, 0.04, 0.0, 0.0, -0.015],
  'K': [[1795.0, 0, 1918.0], [0, 1800.0, 1078.0], [0, 0, 1]]},
 'CAM_3_LEFT': {'fx': 1810.0,
  'fy': 1812.0,
  'cx': 1925.0,
  'cy': 1082.0,
  'D': [-0.27, 0.06, 0.0, 0.0, -0.018],
  'K': [[1810.0, 0, 1925.0], [0, 1812.0, 1082.0], [0, 0, 1]]},
 'CAM_4_RIGHT': {'fx': 1805.0,
  'fy': 1810.0,
  'cx': 1922.0,
  'cy': 1081.0,
  'D': [-0.24, 0.05, 0.0, 0.0, -0.017],
  'K': [[1805.0, 0, 1922.0], [0, 1810.0, 1081.0], [0, 0, 1]]}}

In [ ]:

# load Image Pairs from Folders
stereo_pairs = {
    "STEREO_1": ("left_stereo_1/*.png", "right_stereo_1/*.png"),  # CAM_1 & CAM_2
    "STEREO_2": ("left_stereo_2/*.png", "right_stereo_2/*.png")   # CAM_3 & CAM_4
}

# 3. Detect Court Lines in Both Views
def detect_court_lines(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    edges = cv.Canny(gray, 50, 150, apertureSize=3)
    lines = cv.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=5)
    return lines

# 4. Extract Correspondences from Court Model
def match_court_lines(lines_left, lines_right):
    # Implement court line matching using homography
    pass

# 5. Compute Homography for Each Camera
def compute_homography(lines, court_model):
    H, _ = cv.findHomography(lines, court_model)
    return H

# 6. Stereo Calibration (Find R, T for each stereo pair)
def stereo_calibrate(objpoints, imgpoints_l, imgpoints_r, cam_left, cam_right):
    flags = cv.CALIB_FIX_INTRINSIC
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    ret, _, _, _, _, R, T, _, _ = cv.stereoCalibrate(
        objpoints, imgpoints_l, imgpoints_r, 
        intrinsics[cam_left]["K"], intrinsics[cam_left]["D"], 
        intrinsics[cam_right]["K"], intrinsics[cam_right]["D"], 
        (width, height), criteria=criteria, flags=flags
    )
    return R, T

# 7. Process Each Stereo Pair
extrinsics = {}

for stereo_id, (left_folder, right_folder) in stereo_pairs.items():
    left_images = sorted(glob.glob(left_folder))
    right_images = sorted(glob.glob(right_folder))

    objpoints, imgpoints_l, imgpoints_r = [], [], []

    for left_img_path, right_img_path in zip(left_images, right_images):
        img_l = cv.imread(left_img_path)
        img_r = cv.imread(right_img_path)

        lines_l = detect_court_lines(img_l)
        lines_r = detect_court_lines(img_r)

        # Match lines between views
        court_matches = match_court_lines(lines_l, lines_r)

        # Compute Homography
        H_left = compute_homography(lines_l, court_matches)
        H_right = compute_homography(lines_r, court_matches)

        # Stereo Pair Mapping
        if stereo_id == "STEREO_1":
            cam_left, cam_right = "CAM_1", "CAM_2"
        else:
            cam_left, cam_right = "CAM_3", "CAM_4"

        # Estimate R, T for Stereo Pair
        R, T = stereo_calibrate([court_matches], [lines_l], [lines_r], cam_left, cam_right)
        extrinsics[stereo_id] = {"R": R.tolist(), "T": T.tolist()}

        print(f"{stereo_id} - Rotation Matrix (R):", R)
        print(f"{stereo_id} - Translation Vector (T):", T)

# 8. Save Extrinsic Calibration Data
import json
with open("stereo_extrinsics.json", "w") as f:
    json.dump(extrinsics, f, indent=4)

print("Extrinsic calibration saved.")
